## FRED API

Brian Dew
July 23, 2017

Retrieve data from FRED API using Python 2.7 with the requests package. Use pandas for calculations and analysis. Create a dictionary of relevant results and present it as a table.

#### Import requests and pandas libraries

In [1]:
# Import preliminaries
import requests
import pandas as pd
import config # file with api key

#### Parameters for API request: series and start date

In [2]:
# The URL for the FRED API
base = 'https://api.stlouisfed.org/fred/series/observations?series_id='

# List of FRED series IDs and their description
s_dict = {'CES3000000008': 'Manufacturing AHE, SA', 
          'CES1000000008': 'Mining and Logging AHE, SA',
          'CES4000000008': 'Trade, Transportation, and Utilities AHE, SA',
          'CES2000000008': 'Construction AHE, SA',
          'CES5000000008': 'Information AHE, SA',
          'CES5500000008': 'Financial Activities AHE, SA',
          'CES6000000008': 'Professional and Business Services AHE, SA',
          'CES6500000008': 'Education and Health Services AHE, SA',
          'CES7000000008': 'Leisure and Hospitality AHE, SA',
          'AHETPI': 'Total Private AHE, SA',
          'CWUR0000SA0': 'CPI-Urban Wage Earners and Clerical Workers, SA'
          }

# Include start date, API key from config.py file and file type json
start_date = '1980-01-01'
dates = '&observation_start={}'.format(start_date)
api_key = '&api_key={}'.format(config.fred_key)
ftype = '&file_type=json'

#### Make requests to Fred API

In [3]:
# Fill a new empty dictionary, d, with pandas series from Fred API
d = {}
for k, v in s_dict.iteritems():
    
    # Create unique URL for each Fred series item in s_dict above
    url = '{}{}{}{}{}'.format(base, k, dates, api_key, ftype)
    
    # Request the data from the URL and format as pandas dataframe
    r = pd.DataFrame(requests.get(url).json()['observations'])
    
    # Build dictionary from series
    d[k] = r.set_index(pd.to_datetime(r['date']))['value'].astype(float)
    
# Combine dictionary series into one pandas dataframe
df = pd.DataFrame(d)

In [4]:
df.to_csv('raw_fred_data.csv')

#### Calculate real wage and growth rates

In [5]:
s_list = s_dict.keys()
s_list.remove('CWUR0000SA0')
# New blank dictionary to build html table
latest_table_dict = {}

# Add a title and data to the dict
latest_table_dict['title'] = 'Wages in selected blue collar industries'
latest_table_dict['subtitle'] = 'Average hourly earnings, production and nonsupervisory workers'
latest_table_dict['date'] = 'As of: {}'.format(df.index[-1].strftime('%B %Y'))
sdf = '{}'.format(df.index[0].strftime('%B %Y'))

# Add values to the dict
sd = latest_table_dict['values'] = {}

#### Calculations ####
for s in df[s_list]:
    
    # General measurements
    sd[s] = {'Fred Series': s}
    sd[s]['Name'] = s_dict[s][:-8]
    
    # Nominal average hourly earnings
    sd[s]['Nominal wage'] = '\${:.2f}'.format(df[s][-1])
    col_n = '{}, Nominal'.format(s_dict[s])
    df[col_n] = df[s].round(2)    
    
    # Real wages
    col_r = '{}, Real'.format(s)
    df[col_r] = df[s].multiply(
        df['CWUR0000SA0'][0]).divide(
        df['CWUR0000SA0'])
    sd[s]['Real wage, 1982-1984 dollars'] = '\${:.2f}'.format(df[col_r][-1])
    
    # Annaual percent change of real wage
    col_rch = '{}, Real, 12-mo percent change'.format(s)
    df[col_rch] = df[col_r].pct_change(12).multiply(100)
    sd[s]['Real wage growth rate, %ch over same month previous year'] = '{:.1f}%'.format(df[col_rch][-1])
    
    # Annualized growth rate, past three months
    col_rac = '{}, Real, Annualized growth, last three months'.format(s)
    df[col_rac] = ((df[col_r].rolling(3).mean()/
                    df[col_r].rolling(3).mean().shift(3)
                   )**4 -1) * 100
    sd[s]['Real wage annualized growth rate for last 3 months'] = '{:.1f}%'.format(df[col_rac][-1])
    
    # Calculate max, min, standard deviation, and total change
    sd[s]['Real wage growth rate max'] = '{:.1f}% ({})'.format(df[col_rac].max(), df[col_rac].idxmax().strftime('%B %Y'))
    sd[s]['Real wage growth rate min'] = '{:.1f}% ({})'.format(df[col_rac].min(), df[col_rac].idxmin().strftime('%B %Y'))
    sd[s]['Real wage growth std. dev.'] = '{:.1f}'.format(df[col_rac].std())
    sd[s]['Total change since '+sdf] = '{:.1f}%'.format((df[col_r][-1] / (df[col_r][0])-1) * 100) 

#### Table of Results

In [6]:
print latest_table_dict['title']
print latest_table_dict['subtitle']
print latest_table_dict['date']
pd.DataFrame(sd).T.set_index('Name')

Wages in selected blue collar industries
Average hourly earnings, production and nonsupervisory workers
As of: June 2017


,Fred Series,Nominal wage,Real wage annualized growth rate for last 3 months,Real wage growth rate max,Real wage growth rate min,"Real wage growth rate, %ch over same month previous year",Real wage growth std. dev.,"Real wage, 1982-1984 dollars",Total change since January 1980
Name,,,,,,,,,
Total Private,AHETPI,\$22.03,0.2%,20.7% (January 2009),-7.5% (May 2011),0.8%,2.9,\$7.22,9.9%
Mining and Logging,CES1000000008,\$27.70,-2.9%,23.2% (November 2008),-10.9% (June 2008),0.5%,4.1,\$9.08,6.0%
Construction,CES2000000008,\$26.64,0.4%,23.3% (January 2009),-8.3% (May 2011),0.8%,3.4,\$8.73,-1.6%
Manufacturing,CES3000000008,\$20.78,0.2%,20.7% (January 2009),-8.2% (May 2011),0.3%,3.0,\$6.81,-0.1%
"Trade, Transportation, and Utilities",CES4000000008,\$19.37,0.0%,18.7% (January 2009),-7.7% (May 2011),0.5%,2.9,\$6.35,-6.6%
Information,CES5000000008,\$30.99,3.0%,17.4% (January 2009),-7.2% (June 2007),1.7%,3.2,\$10.16,11.0%
Financial Activities,CES5500000008,\$26.49,0.2%,18.9% (January 2009),-7.4% (May 2011),-0.0%,3.3,\$8.69,57.1%
Professional and Business Services,CES6000000008,\$25.94,-0.4%,26.6% (January 2009),-7.6% (May 2011),0.5%,3.4,\$8.50,22.4%
Education and Health Services,CES6500000008,\$23.05,0.4%,20.9% (January 2009),-7.6% (July 2008),1.2%,2.9,\$7.56,33.3%
